I Plan to use deep neural network to achieve better accuray. I will take 1 input layer 3 hidden layers and one final output layer.

nx -> 10 -> 10 -> 10 -> 1

In [1]:
import pickle
import matplotlib.pyplot as plt
import torch
import numpy as np
from torchmetrics import ConfusionMatrix
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:

def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data
dataDict = unpickle("../raw/cifar-10-batches-py/data_batch_1")

# print(dataDict.keys())
X=torch.tensor(dataDict[b'data']).T
labels=torch.tensor(dataDict[b'labels']).view(1,-1)
unique_values, counts = torch.unique(labels, return_counts=True)
# Print results
print("Unique values:", unique_values)
print("       Counts:", counts)

X=X.to(dtype=torch.float32)/255  # better to convert the range to 0 to 1 instead of 0 to 255 
X = 2*X-1
Y = (labels==3).to(dtype=torch.float) ## 0 if image is not cat 1 if it is cat

m=len(Y[0])
nx = len(X)

catIndicies=torch.nonzero((labels==3))
print(f"Total Cats: {catIndicies.shape[0]}")
catsData=X[:,catIndicies[:,1]]
catsLabels=Y[:,catIndicies[:,1]]

catsDataR=catsData.repeat(1,8)
catsLabels=catsLabels.repeat(1,8)

X = torch.cat((X,catsDataR),dim=1).to(device=device)
Y = torch.cat((Y,catsLabels),dim=1).to(device=device)
m=len(Y[0])
nx = len(X)

# print(catsData)
print(f"labels:",list(set(dataDict[b'labels'])))
print(f" m: {m}")
print(f"nx: {nx}")
print(f"shape of X: {X.shape}")
print(f"shape of Y: {Y.shape}")

# fig,ax=plt.subplots()
# fig.set_size_inches(2,2)
# randomImageNum = torch.randint(0,m,(1,)).item()
# print(f"Random No: {randomImageNum}")
# ax.imshow((X[:,randomImageNum].view(-1,1024).T.view(32,32,3)+1)/2)
# ax.set_title(f"{Y[0,randomImageNum]}")
# ax.axis("off")

Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
       Counts: tensor([1005,  974, 1032, 1016,  999,  937, 1030, 1001, 1025,  981])
Total Cats: 1016
labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
 m: 18128
nx: 3072
shape of X: torch.Size([3072, 18128])
shape of Y: torch.Size([1, 18128])


In [3]:
def loss(y,A):
    # return (y*y_hat)**2
    return -(y*torch.log(A)+(1-y)*torch.log(1-A))

In [4]:
neurons = [nx,128,64,32,1]

print(f" m: {m}")
print(f"nx: {nx}")
print(f"shape of X: {X.shape}")
print(f"shape of Y: {Y.shape}")

## initialize weights and bias matrx
w0 = torch.rand(size=(neurons[0], neurons[1]),device=device) * 0.01
w1 = torch.rand(size=(neurons[1], neurons[2]),device=device) * 0.01
w2 = torch.rand(size=(neurons[2], neurons[3]),device=device) * 0.01
w3 = torch.rand(size=(neurons[3], neurons[4]),device=device) * 0.01
b0 = torch.rand(size=(neurons[1],1),device=device)*1.0
b1 = torch.rand(size=(neurons[2],1),device=device)*1.0
b2 = torch.rand(size=(neurons[3],1),device=device)*1.0
b3 = torch.rand(size=(neurons[4],1),device=device)*1.0

# Always put this at end 
w0.requires_grad_(True)
w1.requires_grad_(True)
w2.requires_grad_(True)
w3.requires_grad_(True)
b0.requires_grad_(True)
b1.requires_grad_(True)
b2.requires_grad_(True)
b3.requires_grad_(True)

z1 = w0.T @ X +b0
A1 = torch.relu(z1)

z2 = w1.T@A1 + b1
A2 = torch.relu(z2)

z3 = w2.T@A2 + b2
A3 = torch.relu(z3)

z4 = w3.T@A3 + b3
A4 = torch.sigmoid(z4)


L = loss(Y,A4)
J = torch.sum(L)/m
print(f"value of J: {J}")

 m: 18128
nx: 3072
shape of X: torch.Size([3072, 18128])
shape of Y: torch.Size([1, 18128])
value of J: 0.7379055619239807


|                  | **Actual Positive** | **Actual Negative** |
|------------------|---------------------|---------------------|
| **Predicted Positive** | True Positive (TP)     | False Positive (FP)    |
| **Predicted Negative** | False Negative (FN)    | True Negative (TN)     |


In [5]:
alpha=0.01
confmat = ConfusionMatrix(task="binary",num_classes=2)
for i in range(10000):
    J.backward()
    with torch.no_grad():  # Disable gradient tracking during update
        w0 -= alpha * w0.grad
        b0 -= alpha * b0.grad
        
        w1 -= alpha * w1.grad
        b1 -= alpha * b1.grad

        w2 -= alpha * w2.grad
        b2 -= alpha * b2.grad

        w3 -= alpha * w3.grad
        b3 -= alpha * b3.grad
        

        w0.grad.zero_()
        b0.grad.zero_()
        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()
        w3.grad.zero_()
        b3.grad.zero_()

    z1 = w0.T @ X +b0
    A1 = torch.relu(z1)

    z2 = w1.T@A1 + b1
    A2 = torch.relu(z2)

    z3 = w2.T@A2 + b2
    A3 = torch.relu(z3)

    z4 = w3.T@A3 + b3
    A4 = torch.sigmoid(z4)

    L = loss(Y,A4)
    J = torch.sum(L)/m
    if i%200 == 0:
        torch.save({
            'w0': w0, 
            'w1': w1,
            'w2': w2,
            'w3': w3,
            'b0': b0, 
            'b1': b1,
            'b2': b2,
            'b3': b3,
        }, f'./wbs/02-WsAndBs-{i}.pt')

        Y_cpu=Y.cpu()
        A4_cpu=A4.cpu()

        cm = confmat(Y_cpu, (A4_cpu>0.5).float())
        accuracy = (cm[0,0]+cm[1,1])/torch.sum(cm)
        print(f"Iteration: {i:<3d} Cost: {J:<6.2f} Accucary:{accuracy:.2f}")
        print(f"minA: {torch.min(A4).item():0.2f}, maxA {torch.max(A4).item():0.2f}")
        
        print(f"TN:{cm[0,0]:>4}, FP:{cm[0,1]:>4}")
        print(f"FN:{cm[1,0]:>4}, TP:{cm[1,1]:>4}\n")

Iteration: 0   Cost: 0.73   Accucary:0.50
minA: 0.63, maxA 0.70
TN:   0, FP:   0
FN:8984, TP:9144

Iteration: 200 Cost: 0.69   Accucary:0.53
minA: 0.25, maxA 0.53
TN:2480, FP:1953
FN:6504, TP:7191

Iteration: 400 Cost: 0.69   Accucary:0.53
minA: 0.27, maxA 0.53
TN:2537, FP:1989
FN:6447, TP:7155

Iteration: 600 Cost: 0.69   Accucary:0.53
minA: 0.29, maxA 0.53
TN:2601, FP:2052
FN:6383, TP:7092

Iteration: 800 Cost: 0.69   Accucary:0.54
minA: 0.29, maxA 0.53
TN:2622, FP:2052
FN:6362, TP:7092

Iteration: 1000 Cost: 0.69   Accucary:0.53
minA: 0.30, maxA 0.52
TN:2638, FP:2106
FN:6346, TP:7038

Iteration: 1200 Cost: 0.69   Accucary:0.53
minA: 0.30, maxA 0.52
TN:2648, FP:2106
FN:6336, TP:7038

Iteration: 1400 Cost: 0.69   Accucary:0.53
minA: 0.30, maxA 0.52
TN:2654, FP:2106
FN:6330, TP:7038

Iteration: 1600 Cost: 0.69   Accucary:0.53
minA: 0.30, maxA 0.52
TN:2657, FP:2124
FN:6327, TP:7020

Iteration: 1800 Cost: 0.69   Accucary:0.53
minA: 0.30, maxA 0.52
TN:2661, FP:2124
FN:6323, TP:7020

Itera